In [1]:
from sklearn.externals import joblib

import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from scipy import sparse
import gzip, re, string

import warnings
warnings.filterwarnings('ignore')

if __name__ == "__main__":
    
    def parse(path):
        g = gzip.open(path, 'rb')
        for l in g:
                yield eval(l)

    def getDF(path):
        i = 0
        df = {}
        for d in parse(path):
                df[i] = d
                i += 1
        return pd.DataFrame.from_dict(df, orient='index')
    
    def pr(x, y_i, y):
        p = x[y==y_i].sum(0)
        return (p+1) / ((y==y_i).sum()+1)

    df = getDF('reviews_Digital_Music_5.json.gz')
    df['reviewText'].fillna("unknown", inplace=True)

    treated_text = []
    for index, row in df.iterrows():
        treated_text.append(re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])').sub(r' \1 ', row['reviewText']))
    
    vec = TfidfVectorizer(ngram_range=(1,2),
            min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
            smooth_idf=1, sublinear_tf=1 )
    x = vec.fit_transform(treated_text)
    y = (df['overall'] > 3).astype(float).values
    
    r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
    clf = LogisticRegression(C = 10, dual=True, penalty = "l2", n_jobs = -1)
    x_nb = x.multiply(r)
    clf.fit(x_nb, y)

In [2]:
a = "It appears the “old Taylor” can come to the phone after all. For anyone hoping for a more conventional Taylor Swift single from her forthcoming sixth album, “Reputation,” the third time's the charm, as “Gorgeous” finally provides some of the conventional pleasures that only a pop song about falling deeply in crush can."

In [3]:
review = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])').sub(r' \1 ', a)
review = vec.transform([review])
my_prediction = clf.predict(review.multiply(r))

In [7]:
my_prediction

array([ 1.])

In [8]:
int(my_prediction[0])

1

In [13]:
clf.predict_proba(review.multiply(r))

array([[ 0.06320946,  0.93679054]])

In [24]:
b = "very bad. I hate it"

In [26]:
review = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])').sub(r' \1 ', b)
review = vec.transform([review])
my_prediction = clf.predict(review.multiply(r))

In [27]:
my_prediction

array([ 0.])

In [28]:
my_proba = clf.predict_proba(review.multiply(r))

In [29]:
my_proba[0]

array([ 0.58347723,  0.41652277])

In [34]:
df.loc[30, 'reviewText']

'In high school "Stroke You Up" was the jam, so I went downtown and bought the cassette. Let me tell you something: It took me 2 days to listen to it because it was utterly boring. Even "Stroke You Up" became unbearable to listen to. No funk, no groove- just lukewarm soul. That was the first *and* the last time I listened to "Changing Faces." The two chicks who made up "Changing Faces" had airy, monotonous vocals- no edge to their voices at all. Even the album cover looked boring. These chicks had no style or any type of pizzazz at all. Skip this at all costs and get Zhane.'

In [35]:
df.loc[df['overall'] == 1, 'reviewText']

30       In high school "Stroke You Up" was the jam, so...
86       I never understood what's the BIG deal behind ...
94       It's now very well known that the labelling on...
95       I am very disapointed! After looking for this ...
167      I love the Beach Boys as much as anyone.  Let ...
382      As a child and young adult when I tended to be...
427      The commercial failure of KRS1s didactic venom...
690      Before I get any hate mail, I would like to st...
695      It's amazing how these guys keep fooling peopl...
696      Talent-Less in every sense of the word.The mus...
700      I'm not gonna say that I expected better of th...
807      This band is truly terrible. First off, the mu...
808      Punk has had a long and proud tradition of bei...
938      Apparently, a lot of modern young musicians ha...
1063     I don't know. I loved EC and I loved country.....
1137     This is my 2nd time out for Baez's second albu...
1166     Fantastic Voyage is the only good song on this.